# 🎵 Zonos TTS - 구글 코랩

Zonos Text-to-Speech 모델을 구글 코랩에서 사용하는 예제입니다.

## 공식 문서 기준 정확한 구현
- **make_cond_dict** 파라미터: 공식 문서와 100% 일치
- **감정 벡터**: [0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077]
- **speaking_rate**: 0-40 (30=very fast, 10=slow)
- **pitch_std**: 0-400 (20-45=normal, 60-150=expressive)
- **fmax**: 22050 (44.1kHz) 또는 24000 (48kHz)
- **unconditional_keys**: 기본값 {"vqscore_8", "dnsmos_ovrl"}


In [ ]:
# 1. 시스템 의존성 설치 (eSpeak-ng)
!apt-get update
!apt-get install -y espeak-ng

# 2. 필요한 라이브러리 설치
!pip install torch torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install transformers gradio

# 3. Zonos 저장소 클론
!git clone https://github.com/Zyphra/Zonos.git
%cd Zonos

# 4. Zonos 설치
!pip install -e .


In [ ]:
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
from zonos.utils import DEFAULT_DEVICE as device

# 모델 로드 (Transformer 버전 사용)
print("모델 로딩 중...")
model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)
print("모델 로딩 완료!")

# 기본 샘플 실행 (공식 문서 기본값 사용)
text = "Hello, this is a test of Zonos text-to-speech in Google Colab!"
cond_dict = make_cond_dict(
    text=text, 
    speaker=None, 
    language="en-us",
    # 공식 문서 기본값 사용
    emotion=[0.3077, 0.0256, 0.0256, 0.0256, 0.0256, 0.0256, 0.2564, 0.3077],
    speaking_rate=15.0,
    pitch_std=20.0,
    fmax=22050.0
)
conditioning = model.prepare_conditioning(cond_dict)
print("음성 생성 중...")
codes = model.generate(conditioning)
wavs = model.autoencoder.decode(codes).cpu()
output_file = "sample.wav"
torchaudio.save(output_file, wavs[0], model.autoencoder.sampling_rate)
print(f"✅ 음성 파일이 {output_file}에 저장되었습니다!")


In [ ]:
# 구글 코랩용 Gradio 인터페이스 (공식 문서 기준 정확한 구현)
import gradio as gr
import torch
import torchaudio
from zonos.model import Zonos
from zonos.conditioning import make_cond_dict
from zonos.utils import DEFAULT_DEVICE as device
import gc

# 전역 모델 변수 확인 및 로드
print("🔍 모델 상태 확인 중...")
if 'model' not in globals():
    print("⚠️ 모델이 로드되지 않았습니다. Gradio에서 모델을 로드합니다...")
    try:
        model = Zonos.from_pretrained("Zyphra/Zonos-v0.1-transformer", device=device)
        print("✅ Gradio에서 모델 로드 완료!")
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        model = None
else:
    print("✅ 모델이 이미 로드되어 있습니다.")

# TTS 함수 정의 (공식 문서 기준 정확한 파라미터)
def generate_speech(text, language="en-us", happiness=0.3077, sadness=0.0256, disgust=0.0256, fear=0.0256, surprise=0.0256, anger=0.0256, other=0.2564, neutral=0.3077, emotion_unconditional=False, speaking_rate=15.0, pitch_std=20.0, fmax=22050.0, speaker_unconditional=False, fmax_unconditional=False, pitch_unconditional=False, rate_unconditional=False, seed=None, audio_file=None):
    try:
        print(f"🔍 디버깅: 텍스트='{text}', 언어='{language}', 시드='{seed}', 오디오파일='{audio_file}'")

        # 모델 확인
        if model is None:
            return "❌ 모델이 로드되지 않았습니다. 페이지를 새로고침하고 다시 시도해주세요.", "오류 발생"

        print("✅ 모델 확인 완료")

        # 시드 설정
        if seed is not None:
            torch.manual_seed(seed)
            used_seed = seed
            print(f"🎲 시드 설정: {seed}")
        else:
            # 랜덤 시드 생성
            used_seed = torch.randint(0, 2**32 - 1, (1,)).item()
            torch.manual_seed(used_seed)
            print(f"🎲 랜덤 시드 생성: {used_seed}")

        # 음성 클로닝 처리
        speaker_embedding = None
        if audio_file is not None:
            try:
                print("🎤 음성 클로닝 처리 중...")
                wav, sampling_rate = torchaudio.load(audio_file)
                speaker_embedding = model.make_speaker_embedding(wav, sampling_rate)
                print("✅ 스피커 임베딩 생성 완료")
            except Exception as e:
                print(f"⚠️ 음성 클로닝 실패: {e}")
                print("기본 스피커로 진행합니다.")

        # 감정 벡터 구성 (공식 문서 기본값)
        emotion_vector = [happiness, sadness, disgust, fear, surprise, anger, other, neutral]
        print(f"🎭 감정 벡터: {emotion_vector}")

        # 무조건적 설정 구성 (공식 문서 기본값: {"vqscore_8", "dnsmos_ovrl"})
        unconditional_keys = {"vqscore_8", "dnsmos_ovrl"}  # 공식 기본값
        
        # 사용자 설정 무조건적 설정
        if emotion_unconditional:
            unconditional_keys.add("emotion")
        if speaker_unconditional:
            unconditional_keys.add("speaker")
        if fmax_unconditional:
            unconditional_keys.add("fmax")
        if pitch_unconditional:
            unconditional_keys.add("pitch_std")
        if rate_unconditional:
            unconditional_keys.add("speaking_rate")
        
        print(f"🔓 무조건적 설정: {unconditional_keys}")

        # 텍스트 전처리
        print(f"📝 원본 텍스트: '{text}'")
        processed_text = text.strip()
        print(f"📝 처리된 텍스트: '{processed_text}'")

        # 조건 설정 (공식 문서 기준)
        print("🔧 조건 설정 중...")
        cond_dict = make_cond_dict(
            text=processed_text,
            speaker=speaker_embedding,
            language=language,
            emotion=emotion_vector,
            speaking_rate=speaking_rate,
            pitch_std=pitch_std,
            fmax=fmax,
            unconditional_keys=unconditional_keys
        )
        print("✅ 조건 설정 완료")
        print(f"🔍 espeak 텍스트: {cond_dict.get('espeak', 'N/A')}")

        # 조건 준비
        print("🎯 조건 준비 중...")
        conditioning = model.prepare_conditioning(cond_dict)
        print("✅ 조건 준비 완료")

        # 음성 생성 (공식 코드와 동일한 파라미터 사용)
        print("🎵 음성 생성 중... (시간이 걸릴 수 있습니다)")
        codes = model.generate(
            conditioning,
            cfg_scale=2.0,  # 공식 기본값
            batch_size=1,
            sampling_params=dict(
                top_p=0.0,
                top_k=0,
                min_p=0.0,
                linear=0.5,
                conf=0.40,
                quad=0.00
            )
        )
        print("✅ 음성 생성 완료")

        # 디코딩
        print("🔊 디코딩 중...")
        wavs = model.autoencoder.decode(codes).cpu()
        print("✅ 디코딩 완료")

        # 결과 저장
        output_file = "generated_speech.wav"
        torchaudio.save(output_file, wavs[0], model.autoencoder.sampling_rate)
        print(f"✅ 음성 파일 저장 완료: {output_file}")

        # 메모리 정리
        torch.cuda.empty_cache()
        gc.collect()

        return output_file, f"시드: {used_seed}"

    except Exception as e:
        error_msg = f"❌ 오류 발생: {str(e)}"
        print(error_msg)
        import traceback
        print("상세 오류:")
        traceback.print_exc()
        return error_msg, "오류 발생"

# Gradio 인터페이스 생성 (공식 데모와 동일한 구조)
with gr.Blocks(title="Zonos TTS - 구글 코랩") as demo:
    gr.Markdown("# 🎵 Zonos TTS - 구글 코랩")
    gr.Markdown("텍스트를 음성으로 변환하는 고급 TTS 시스템입니다.")

    with gr.Row():
        with gr.Column():
            text_input = gr.Textbox(
                label="텍스트 입력",
                placeholder="여기에 변환할 텍스트를 입력하세요...",
                lines=3
            )

            language = gr.Dropdown(
                choices=["en-us", "ko", "ja", "zh", "fr", "de"],
                value="en-us",
                label="언어 선택"
            )

            with gr.Accordion("🎭 감정 설정", open=False):
                gr.Markdown("감정 벡터 (0.0-1.0): Happiness, Sadness, Disgust, Fear, Surprise, Anger, Other, Neutral")
                
                happiness = gr.Slider(0.0, 1.0, value=0.3077, step=0.0001, label="Happiness (행복)")
                sadness = gr.Slider(0.0, 1.0, value=0.0256, step=0.0001, label="Sadness (슬픔)")
                disgust = gr.Slider(0.0, 1.0, value=0.0256, step=0.0001, label="Disgust (혐오)")
                fear = gr.Slider(0.0, 1.0, value=0.0256, step=0.0001, label="Fear (두려움)")
                surprise = gr.Slider(0.0, 1.0, value=0.0256, step=0.0001, label="Surprise (놀람)")
                anger = gr.Slider(0.0, 1.0, value=0.0256, step=0.0001, label="Anger (분노)")
                other = gr.Slider(0.0, 1.0, value=0.2564, step=0.0001, label="Other (기타)")
                neutral = gr.Slider(0.0, 1.0, value=0.3077, step=0.0001, label="Neutral (중립)")
            
            speaking_rate = gr.Slider(
                minimum=0.0,
                maximum=40.0,
                value=15.0,
                step=1.0,
                label="Speaking Rate (0-40 phonemes per minute, 30=very fast, 10=slow)"
            )

            pitch_std = gr.Slider(
                minimum=0.0,
                maximum=400.0,
                value=20.0,
                step=5.0,
                label="Pitch Std (0-400, 20-45=normal, 60-150=expressive, higher=crazier)"
            )

            fmax = gr.Slider(
                minimum=0, maximum=24000, value=22050, step=50,
                label="Fmax (Hz) - 22050=44.1kHz, 24000=48kHz, 22050 for voice cloning"
            )

            seed = gr.Number(
                value=None,
                label="Seed (for reproducible results, leave empty for random)",
                precision=0
            )

            audio_file = gr.Audio(
                label="Voice Cloning Audio (10-30 seconds recommended, optional)",
                type="filepath"
            )

            with gr.Accordion("⚙️ Advanced Settings", open=False):
                gr.Markdown("### Unconditional Keys")
                gr.Markdown("Check to make the parameter unconditional (auto-filled by model)")
                speaker_unconditional = gr.Checkbox(label="speaker", value=False)
                emotion_unconditional = gr.Checkbox(label="emotion", value=False)
                fmax_unconditional = gr.Checkbox(label="fmax", value=False)
                pitch_unconditional = gr.Checkbox(label="pitch_std", value=False)
                rate_unconditional = gr.Checkbox(label="speaking_rate", value=False)

            generate_btn = gr.Button("🎵 Generate Speech", variant="primary")

        with gr.Column():
            audio_output = gr.Audio(label="Generated Speech", type="filepath")
            current_seed = gr.Textbox(
                label="Current Seed",
                value="Not generated yet",
                interactive=False,
                info="Copy this seed to reproduce the same result later"
            )

            # GPU 정보 표시
            gpu_info = ""
            if torch.cuda.is_available():
                gpu_name = torch.cuda.get_device_name(0)
                gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
                gpu_info = f"🖥️ **GPU**: {gpu_name} ({gpu_memory:.1f} GB)"
            else:
                gpu_info = "❌ **GPU**: Not available (CPU mode)"
            
            gr.Markdown(f"""
            ### 🖥️ System Info:
            {gpu_info}
            
            ### 🌐 Supported Languages:
            - **English (en-us)**: Hello, how are you?
            - **한국어 (ko)**: 안녕하세요, 반갑습니다!
            - **日本語 (ja)**: こんにちは、元気ですか？
            - **中文 (zh)**: 你好，最近怎么样？
            - **Français (fr)**: Bonjour, comment allez-vous ?
            - **Deutsch (de)**: Hallo, wie geht es Ihnen?

            ### 🎯 Key Features:
            - **Seed Control**: Use same seed for reproducible results
            - **Voice Cloning**: Upload 10-30s audio for zero-shot voice cloning
            - **Emotion Control**: Fine-tune 8 emotion dimensions
            - **Speech Parameters**: Adjust speaking rate, pitch variation, frequency range

            ### 📖 Usage:
            1. Enter text to convert
            2. Select language and adjust emotions
            3. Tune speaking rate and pitch
            4. Set seed for reproducibility (optional)
            5. Upload audio for voice cloning (optional)
            6. Click "Generate Speech"
            7. Play or download the generated audio
            """)

    # 이벤트 연결
    generate_btn.click(
        fn=generate_speech,
        inputs=[text_input, language, happiness, sadness, disgust, fear, surprise, anger, other, neutral, emotion_unconditional, speaking_rate, pitch_std, fmax, speaker_unconditional, fmax_unconditional, pitch_unconditional, rate_unconditional, seed, audio_file],
        outputs=[audio_output, current_seed]
    )

# 인터페이스 실행 (퍼블릭 링크 생성)
print("=== Gradio 웹 인터페이스 실행 ===")
print("퍼블릭 링크가 생성되면 브라우저에서 접속할 수 있습니다.")

demo.launch(share=True)
